In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

In [2]:
data = pd.read_excel('RawData_ComesF.xlsx',"Planilha2",index_col=False)

In [3]:
data.head()

,vague,num questio,CONTRÔLE Num Questio versus recrut Tel,Type de questionnaire,Age du sujet de l'enquête,Sexe du sujet de l'enquête,Lien du répondant pour l'enfant <15 ans Sujet de l'enquête,Age du répondant pour l'enfant <15 ans Sujet de l'enquête,Sexe du répondant pour l'enfant <15 ans Sujet de l'enquête,Nombre de personnes au foyer,...,Unnamed: 48,Unnamed: 49,A ou non plus de 20 contacts professionnels,Nombre d'étudiants dans la classe,Etudiant qui mange à la cantine ?,jour1mois1,CONTRÔLE validation jour1mois1 en période vague 1 ou 2,joursemaine1,Vacances jour1mois1,Nombre de contacts saisis JOUR 1
0,2,15218,1,1,38,2,NaN,NaN,NaN,2,...,0,0,NaN,NaN,NaN,105.0,1.0,2.0,1,2
1,2,3996,1,0,0,2,1.0,29.0,1.0,4,...,0,0,NaN,NaN,NaN,2004.0,1.0,5.0,1,6
2,2,14409,1,1,18,2,NaN,NaN,NaN,4,...,0,0,NaN,3.0,3.0,205.0,1.0,3.0,1,11
3,2,952,1,1,74,1,NaN,NaN,NaN,2,...,0,0,NaN,NaN,NaN,2904.0,1.0,7.0,2,6
4,2,12382,1,1,71,1,NaN,NaN,NaN,1,...,0,0,NaN,NaN,NaN,2004.0,1.0,5.0,2,3


In [4]:
idade = [i for i in data.columns if('AGE' in i)]
idades = [list(i) for i in data[idade].fillna(0).values]
df = {
    'idades': idades
}

In [52]:
mode = [i for i in data.columns if('mode(s)' in i)][0]
mode = list(data.columns).index(mode)
mode = data.columns[mode:mode+3]

In [53]:
modes = [list(i) for i in data[mode].fillna(0).values]
df['modes1'] = modes

In [56]:
df['modes2']

[[0, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 1, 1],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 1],
 [1, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 0],
 [1, 0, 1],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 1],
 [1, 0, 1],
 [0, 1, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 1],
 [1, 1, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 1],
 [1, 0, 1],
 [1, 0, 1],
 [1, 0, 1],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 1],
 [1, 0, 1],
 [0, 0, 0],
 [0, 1, 1],
 [1, 0, 0],
 [1, 1, 1],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 0],
 [1, 1, 0],
 [0, 0, 0],
 [1, 0, 1],
 [0, 1, 1],
 [0, 0, 0],
 [1, 0, 0],
 [1, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 1],
 [1,

In [5]:
serv = Service(ChromeDriverManager().install())

In [6]:
coluna = []
option = webdriver.ChromeOptions()
option.add_argument('headless')
driver = webdriver.Chrome(service = serv,options=option)

driver.get("https://translate.google.com.br/?hl=pt-BR")

traduz = '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[1]/span/span/div/textarea'

time.sleep(5)

lingua = '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[1]/c-wiz/div[1]/c-wiz/div[2]/button'

driver.find_element(By.XPATH, lingua).click()

lingua = '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[1]/c-wiz/div[2]/c-wiz/div[1]/div/div[3]/div/div[3]/div[45]/div[2]'

driver.find_element(By.XPATH, lingua).click()

for i in data.columns:

    driver.find_element(By.XPATH, traduz).send_keys(i)
    traduzido = '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[2]/div[8]/div/div[1]/span[1]/span/span'


    time.sleep(5)
    coluna.append(driver.find_element(By.XPATH, traduzido).text)

    driver.find_element(By.XPATH, traduz).clear()

driver.quit()

In [8]:
data.columns = coluna

In [9]:
data

,vago,questão de número,Controle de Num Questio versus recruta como,Tipo de questionário,Idade do assunto da pesquisa,Sexo do assunto da investigação,Link do entrevistado para a criança <15 anos sujeito à pesquisa,Idade do entrevistado para a criança <15 anos de investigação,Entrevistado para a criança <15 anos sujeitos à pesquisa,Número de famílias,...,Unnam: 48,Unnam: 49,A ou não mais que 20 contatos profissionais,Número de alunos na sala de aula,Aluno que come na cantina?,DIA1MOIS1,Controle de validação Day1Mois1 em vago 1 ou 2,Daysemaine1,Feriados dia1 meses1,Número de contatos inseridos no dia 1
0,2,15218,1,1,38,2,NaN,NaN,NaN,2,...,0,0,NaN,NaN,NaN,105.0,1.0,2.0,1,2
1,2,3996,1,0,0,2,1.0,29.0,1.0,4,...,0,0,NaN,NaN,NaN,2004.0,1.0,5.0,1,6
2,2,14409,1,1,18,2,NaN,NaN,NaN,4,...,0,0,NaN,3.0,3.0,205.0,1.0,3.0,1,11
3,2,952,1,1,74,1,NaN,NaN,NaN,2,...,0,0,NaN,NaN,NaN,2904.0,1.0,7.0,2,6
4,2,12382,1,1,71,1,NaN,NaN,NaN,1,...,0,0,NaN,NaN,NaN,2004.0,1.0,5.0,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,2,7528,1,1,12,2,NaN,NaN,NaN,4,...,0,0,NaN,2.0,3.0,2404.0,1.0,2.0,2,10
2029,2,7982,1,0,2,1,1.0,29.0,1.0,4,...,0,0,NaN,NaN,NaN,2204.0,1.0,7.0,1,10
2030,2,8023,1,1,22,1,NaN,NaN,NaN,5,...,0,0,NaN,NaN,NaN,1704.0,1.0,2.0,1,9
2031,2,7383,1,0,0,1,1.0,33.0,1.0,5,...,0,0,NaN,NaN,NaN,2404.0,1.0,2.0,2,8
